# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.10000000149011612
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.09000000171363354


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
env_info = env.reset(train_mode=True)[brain_name]

In [7]:
from ou_noise import OUNoise
from utils import hard_update, soft_update
from replay_buffer import ReplayBuffer

import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class Actor(nn.Module):
    """Actor (Policy) Model."""
    def __init__(self, 
                 in_actor,
                 hidden_in_actor=256, 
                 hidden_out_actor=256,
                 out_actor=2,
                 leak=0.01, 
                 seed=42):
        """ Initialize parameters and build model.
        Params
        ======
            in_actor (int): Size of state tensor
            hidden_in_actor (int): Number of hidden units in FC layer 1
            hidden_out_actor (int): Number of hidden units in FC layer 2           
            out_actor (int): Size of action tensor
            seed (int): Random seed                    
            leak (float): the leak rate for leaky ReLU, i.e. the alpha in (x < 0) * alpha * x + (x >= 0) * x
        """
        super(Actor, self).__init__()
        self.leak = leak
        self.seed = torch.manual_seed(seed)

        self.fc1 = nn.Linear(in_actor, hidden_in_actor)
        self.fc2 = nn.Linear(hidden_in_actor, hidden_out_actor)
        self.fc3 = nn.Linear(hidden_out_actor, out_actor)

        self.bn = nn.BatchNorm1d(in_actor)
        self.reset_parameters()

    def reset_parameters(self):
        """Initialize FC layers followed by leaky ReLU using Kaiming He's (2015) approach.
        Source: https://arxiv.org/pdf/1502.01852v1.pdf
        For more info see here:
            https://www.jefkine.com/deep/2016/08/08/initialization-of-deep-networks-case-of-rectifiers/
        """
        torch.nn.init.kaiming_normal_(
            self.fc1.weight.data,
            a=self.leak, 
            nonlinearity='leaky_relu',
            mode='fan_in')
        torch.nn.init.kaiming_normal_(self.fc2.weight.data,
                                      a=self.leak,
                                      nonlinearity='leaky_relu',
                                      mode='fan_in')
        torch.nn.init.uniform_(self.fc3.weight.data,
                               -3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
#         state = self.bn(state)
        x = F.leaky_relu(self.fc1(state), negative_slope=self.leak)
        x = F.leaky_relu(self.fc2(x), negative_slope=self.leak)
        x =  torch.tanh(self.fc3(x))
        return x

In [31]:
class Critic(nn.Module):
    """Critic (Value) Model."""
    def __init__(self,
                 in_critic,
                 hidden_in_critic=256,
                 hidden_out_critic=256,
                 out_critic=1, 
                 leak=0.01, 
                 seed=42):
        """Initialize parameters and build model.
        Params
        ======
            in_critic (int): Dimension of each state            
            hidden_in_critic (int): Number of nodes in the first hidden layer
            hidden_out_critic (int): Number of nodes in the second hidden layer
            out_critic (int): Dimension of each action
            leak (float): Leakiness of leaky ReLU
        """
        super(Critic, self).__init__()
        self.leak = leak
        self.seed = torch.manual_seed(seed)
        self.bn = nn.BatchNorm1d(in_critic)
        self.fc1 = nn.Linear(in_critic, hidden_in_critic)
        self.fc2 = nn.Linear(hidden_in_critic, hidden_out_critic)
        self.fc3 = nn.Linear(hidden_out_critic, out_critic)
        self.reset_parameters()

    def reset_parameters(self):
        """Initialize FC layers followed by leaky ReLU using Kaiming He's (2015) approach.
        Source: https://arxiv.org/pdf/1502.01852v1.pdf
        For more info see here:
            https://www.jefkine.com/deep/2016/08/08/initialization-of-deep-networks-case-of-rectifiers/
        """
        torch.nn.init.kaiming_normal_(self.fc1.weight.data, a=self.leak,
                                      nonlinearity='leaky_relu', mode='fan_in')
        torch.nn.init.kaiming_normal_(self.fc2.weight.data, a=self.leak,
                                      nonlinearity='leaky_relu', mode='fan_in')
        torch.nn.init.uniform_(self.fc3.weight.data, -3e-3, 3e-3)

    def forward(self, state):
        """ Build a critic (value) network that maps (state, action) pairs -> Q-values."""
#         state = self.bn(state)
        x = F.leaky_relu(self.fc1(state), negative_slope=self.leak)
        x = F.leaky_relu(self.fc2(x), negative_slope=self.leak)
        x =  self.fc3(x)
        return x

In [32]:
class DDPGAgent:
    def __init__(self, 
                 in_actor, 
                 hidden_in_actor,
                 hidden_out_actor,
                 out_actor, 
                 in_critic,
                 hidden_in_critic,
                 hidden_out_critic,
                 out_critic,
                 device,
                 relu_leak=1e-2,
                 seed=42,
                 lr_actor=1.0e-2, 
                 lr_critic=1.0e-2):
        super(DDPGAgent, self).__init__()
        self.device=device
        self.actor = Actor(
            in_actor,
            hidden_in_actor, 
            hidden_out_actor, 
            out_actor, 
            relu_leak, 
            seed).to(device)
        self.critic = Critic(
            in_critic,
            hidden_in_critic,
            hidden_out_critic, 
            out_critic, 
            relu_leak, 
            seed).to(device)
        self.target_actor = Actor(
            in_actor, 
            hidden_in_actor,
            hidden_out_actor,
            out_actor, 
            relu_leak,
            seed).to(device)
        self.target_critic = Critic(
            in_critic, 
            hidden_in_critic,
            hidden_out_critic,
            out_critic, 
            relu_leak, 
            seed).to(device)

        self.noise = OUNoise(out_actor, seed)
        
        # initialize targets same as original networks
        hard_update(self.target_actor, self.actor)
        hard_update(self.target_critic, self.critic)

        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=lr_critic, weight_decay=1.e-5)   
        
    def act(self, states, noise_mul=0.0):
        self.actor.eval()
        with torch.no_grad():
            actions = self.actor(states).cpu().data.numpy()
        actions += self.noise.sample() * noise_mul
        return np.clip(actions, -1, 1)        
    
    def target_act(self, states, noise_mul=0.0):
        self.target_actor.eval()
        with torch.no_grad():
            actions = self.target_actor(states).cpu().data.numpy()
        actions += self.noise.sample() * noise_mul
        return np.clip(actions, -1, 1)
    
    def reset(self):
        self.noise.reset()

In [33]:
class MADDPGAgent:
    def __init__(
        self,
        num_agents,
        in_actor, 
        hidden_in_actor,
        hidden_out_actor,
        out_actor, 
        hidden_in_critic,
        hidden_out_critic,
        out_critic,
        device,
        relu_leak=1e-2,
        seed=42,
        lr_actor=1.0e-2, 
        lr_critic=1.0e-2,
        replay_buffer_size=int(1e6),
        batch_size=64,
        gamma=0.99,
        tau=1e-3,
        update_every=1,
        num_updates=1
    ):        
        in_critic = in_actor * num_agents + out_actor
        self.agents = []
        self.num_agents = num_agents
        self.out_actor = out_actor
        self.replay_buffer = ReplayBuffer(num_agents * out_actor, replay_buffer_size, batch_size, seed, device)
        self.batch_size = batch_size
        self.gamma = gamma
        self.tau = tau
        self.device = device
        self.steps = 0
        self.update_every = update_every
        self.num_updates = num_updates
        self.noise_scale = 1.0
        for _ in range(num_agents):
            self.agents.append(
                DDPGAgent(
                    in_actor, hidden_in_actor, hidden_out_actor, out_actor,
                    in_critic, hidden_in_critic, hidden_out_critic, out_critic, 
                    device, relu_leak, seed, lr_actor, lr_critic)
            )          

    def act(self, states, noise_mul=0.0):
        """Returns actions for given state as per current policy."""
        actions = np.zeros((self.num_agents, self.out_actor))
        for agent_idx, state in enumerate(states):
            state = torch.from_numpy(state).float().to(self.device)
            if state.dim() < 2:
                state = state.unsqueeze(0)
            curr_agent = self.agents[agent_idx]
            action = curr_agent.act(state, noise_mul)
            actions[agent_idx, :] = action
        return actions
            
    def step(self, states, actions, rewards, next_states, dones):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        self.steps += 1
        self.replay_buffer.add(states, actions, rewards, next_states, dones)
        print(self.replay_buffer.memory[0][0])
        print(self.replay_buffer.memory[0][1])
        print(self.replay_buffer.memory[0][2])
        print(self.replay_buffer.memory[0][3])
        print(self.replay_buffer.memory[0][4])

        if (len(self.replay_buffer) > self.batch_size) and (self.steps % self.update_every == 0):
            # reset steps so we don't overflow at some point
            self.steps = 0
            print("learning")
            for _ in range(self.num_updates):
                experiences = self.replay_buffer.sample()
                self.learn(experiences, self.gamma)
                
    def learn(self, experiences, gamma):
        
        all_agents_next_actions = []
        all_agents_actions = []
        
        for agent_id, agent in enumerate(self.agents):
            
            all_states, all_actions, all_rewards, all_next_states, all_dones = experiences
            agent_idx = torch.tensor([agent_id]).to(self.device)
            states = torch.index_select(all_states, 1, agent_idx)
            actions = torch.index_select(all_actions, 1, agent_idx)
            rewards = torch.index_select(all_rewards, 1, agent_idx)
            next_states = torch.index_select(all_next_states, 1, agent_idx)
            dones = torch.index_select(all_dones, 1, agent_idx)
            
            # Critic (aware of all agents' observations)
            agent.critic_optimizer.zero_grad()
            target_actions = agent.target_act(next_states, self.noise_scale)
            target_actions = torch.from_numpy(target_actions).to(self.device)
#             target_actions = torch.cat(target_actions, dim=1)        
            print("all_next_states.shape={}".format(all_next_states.shape))
            all_next_states = all_next_states.reshape(self.batch_size, -1)
            print("all_next_states.shape={}".format(all_next_states.shape))
            target_actions = target_actions.squeeze()
            print("target_actions.shape={}".format(target_actions.shape))
            target_critic_input = torch.cat((all_next_states.reshape(self.batch_size, -1),target_actions), dim=1)
            with torch.no_grad():
                q_next = agent.target_critic(target_critic_input)
            y = rewards[agent_id].view(-1, 1) + self.gamma * q_next * (1 - done[agent_number].view(-1, 1))
            action = torch.cat(action, dim=1)
            critic_input = torch.cat((all_states.t(), action), dim=1).to(device)
            q = agent.critic(critic_input)
            huber_loss = torch.nn.SmoothL1Loss()
            critic_loss = huber_loss(q, y.detach())
            critic_loss.backward()
            agent.critic_optimizer.step()
            
            # Actor
            
            agent.actor_optimizer.zero_grad()
            
            q_input = [ self.maddpg_agent[i].actor(ob) if i == agent_number \
                       else self.maddpg_agent[i].actor(ob).detach()
                       for i, ob in enumerate(obs) ]                        
            
            q_input = torch.cat(q_input, dim=1)
            q_input2 = torch.cat((obs_full.t(), q_input), dim=1)
            
            actor_loss = -agent.critic(q_input2).mean()
            actor_loss.backward()
            #torch.nn.utils.clip_grad_norm_(agent.actor.parameters(),0.5)
            agent.actor_optimizer.step()
            
            al = actor_loss.cpu().detach().item()
            cl = critic_loss.cpu().detach().item()
            print("Agent {} losses: critic loss: {}, actor loss: {}".format(agent_id, cl, al))        
    
    def reset(self):
        self.noise.reset()
        
    def save_agents(self):
        raise NotImplementedError("Saving is not implemented yet.")


In [34]:
num_agents = 2

in_actor = 24
hidden_in_actor = 128
hidden_out_actor = 128
out_actor = 2

hidden_in_critic = 128
hidden_out_critic = 1
out_critic = 1

lr_actor = 1e-3
lr_critic = 1e-3 
relu_leak = 1e-2
seed = 42

gpu_id = 0
use_gpu = True
device = 'cuda:{}'.format(gpu_id) if use_gpu else 'cpu'

replay_buffer_size = int(1e6)
batch_size = 16
gamma = 0.99
tau = 1e-3
update_every = 1
num_updates = 1

meta_agent = MADDPGAgent(  
    num_agents,
    in_actor, 
    hidden_in_actor,
    hidden_out_actor,
    out_actor, 
    hidden_in_critic,
    hidden_out_critic,
    out_critic,
    device,
    relu_leak,
    seed,
    lr_actor, 
    lr_critic,
    replay_buffer_size,
    batch_size,
    gamma,
    tau,
    update_every,
    num_updates)

In [35]:
env_info = env.reset(train_mode=False)[brain_name]

states = env_info.vector_observations

actions = meta_agent.act(states)
print(actions)

[[ 0.04495187 -0.01474663]
 [ 0.03207987  0.00519489]]


In [36]:
num_episodes = 1000

num_steps_max = 1000

print_every = 10

for i in range(num_episodes):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)

    for j in range(num_steps_max):
        actions = meta_agent.act(states)
        print(actions)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards        
        dones = env_info.local_done
        print("states.shape={}".format(states.shape))
        print("actions.shape={}".format(actions.shape))
        print("rewards.shape={}".format(len(rewards)))
        print("next_states.shape={}".format(next_states.shape))
        print("dones.shape={}".format(len(dones)))
        actions = torch.from_numpy(actions).to(meta_agent.device)
        meta_agent.step(states, actions, rewards, next_states, dones)
        scores += env_info.rewards
        states = next_states
        if np.any(dones):            
            break
            
        if j > batch_size and j % print_every == 0:
            print("episode {}, step {}: done".format(i, j))
            
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

[[ 0.03867292 -0.01440356]
 [ 0.02226311 -0.00378614]]
states.shape=(2, 24)
actions.shape=(2, 2)
rewards.shape=2
next_states.shape=(2, 24)
dones.shape=2
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.14420891 -1.5
  -0.          0.          7.5460577  -2.30871558 -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.74939442 -1.5
   0.          0.         -7.5460577  -2.30871558  0.          0.        ]]
tensor([[ 0.0387, -0.0144],
        [ 0.0223, -0.0038]], device='cuda:0', dtype=torch.float64)
[0.0, -0.009999999776482582]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0. 

states.shape=(2, 24)
actions.shape=(2, 2)
rewards.shape=2
next_states.shape=(2, 24)
dones.shape=2
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.14420891 -1.5
  -0.          0.          7.5460577  -2.30871558 -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.74939442 -1.5
   0.          0.         -7.5460577  -2.30871558  0.          0.        ]]
tensor([[ 0.0387, -0.0144],
        [ 0.0223, -0.0038]], device='cuda:0', dtype=torch.float64)
[0.0, -0.009999999776482582]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.59082413 -1.5
  -0.          0.          6.5

states.shape=(2, 24)
actions.shape=(2, 2)
rewards.shape=2
next_states.shape=(2, 24)
dones.shape=2
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.14420891 -1.5
  -0.          0.          7.5460577  -2.30871558 -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.74939442 -1.5
   0.          0.         -7.5460577  -2.30871558  0.          0.        ]]
tensor([[ 0.0387, -0.0144],
        [ 0.0223, -0.0038]], device='cuda:0', dtype=torch.float64)
[0.0, -0.009999999776482582]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.59082413 -1.5
  -0.          0.          6.5

NameError: name 'done' is not defined

In [ ]:
states.shape